In [13]:
import numpy as np
import pandas as pd
import math
import string
import pickle
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from scipy.cluster.vq import whiten

def get_tf_idf_query_similarity(docs_tfidf, query_tfidf):
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    return cosineSimilarities

# def l2_norm(a):
#     return math.sqrt(np.dot(a, a))

# def cosine_similarity(a, b):
#     return np.dot(a,b) / (l2_norm(a) * l2_norm(b))

sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

stops = stopwords.words('english')
punctuation_str = string.punctuation
symbol_emoji_list = [":)",  ";)",  ":(",  ":\\",  ":|", ":]", ":[",
                     ":-)", ";-)", ":-(", ":-\\", ":-|", ":-[", ";-]",
                     ":D", ":P", ":-x", ":'-(", ":_(", ":o)", "XD", ":'(", ":->",
                     "o_O", "T_T", "^o^", 
                     ":-D", ":-P","B-)", "8-)", ":-o", ":-O", ":-0", ":-s", ":-S"]
# TODO hashtag #
# TODO hmmmmm hhuuuuugg

In [20]:
version = "v2_1"
dev_set_path = '../data/{0}/dev_set_{0}.txt'.format(version)
train_set_path = '../data/{0}/train_set_{0}.txt'.format(version)

train_file = open(train_set_path, 'rb')
train_data = pickle.load(train_file)
dev_data = pd.read_csv(dev_set_path, header=None, sep='\t')
#train_label = (np.array(train_data)[:,0]).astype('int')
dev_label = (np.array(dev_data)[:,0]).astype('int')
#train_sentence_list = train_data[1]
dev_sentence_list = dev_data[1]